<a href="https://colab.research.google.com/github/akshithamaddi/Akshitha_INFO5731_Spring2020/blob/master/Copy_of_INFO5731_Assignment_Three_(7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Four**

In this assignment, you are required to conduct topic modeling, sentiment analysis based on **the dataset you created from assignment three**.

# **Question 1: Topic Modeling**

(30 points). This question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents. Based on the dataset from assignment three, write a python program to **identify the top 10 topics in the dataset**. Before answering this question, please review the materials in lesson 8, especially the code for LDA and LSA. The following information should be reported:

(1) Features (top n-gram phrases) used for topic modeling.

(2) Top 10 clusters for topic modeling.

(3) Summarize and describe the topic for each cluster. 


In [0]:
import pandas as pd
papers = pd.read_csv("Assignment5_File_5731.csv",encoding = "ISO-8859-1")
papers_data = papers['Clean_Title']
papers.head()

,Document_ID,Clean_Title,Sentiment
0,1,viewer actual went tiff wit film didnt want be...,Positive
1,2,outstand movi haunt perform best charact devel...,Positive
2,3,certain peopl relat,Neutral
3,4,perfect everi aspect,Positive
4,5,hype real,Positive


In [0]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = papers.Clean_Title.values.tolist()
data_words = list(sent_to_words(data))
print(data_words)

[['viewer', 'actual', 'went', 'tiff', 'wit', 'film', 'didnt', 'want', 'believ', 'hype', 'absolut', 'masterpiec', 'phoenix', 'certifi', 'legend'], ['outstand', 'movi', 'haunt', 'perform', 'best', 'charact', 'develop', 'ever', 'seen'], ['certain', 'peopl', 'relat'], ['perfect', 'everi', 'aspect'], ['hype', 'real'], ['masterpiec'], ['amaz', 'movi', 'exist'], ['went', 'second', 'time', 'watch'], ['psycholog', 'studi', 'rather', 'superhero', 'flick'], ['joaquin', 'oscar', 'joker', 'best', 'dark', 'suspens', 'thriller', 'darker', 'dark', 'knight'], ['venic', 'review'], ['represint', 'real', 'life', 'joker'], ['final', 'real', 'movi'], ['good', 'lord'], ['spoon', 'feed', 'cgi', 'fuel', 'faux', 'drama'], ['oscar', 'phoenix'], ['critic', 'useless'], ['joker', 'endgam'], ['would', 'call', 'masterpiec'], ['one', 'best', 'act', 'perform', 'ive', 'ever', 'seen'], ['dont', 'forget', 'smile'], ['ok', 'film'], ['extrem', 'overr'], ['believ', 'hype'], ['masterpiec'], ['great', 'dont', 'know'], ['that',

In [0]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import spacy
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized)

[['go', 'want', 'believ', 'hype'], ['perform', 'good', 'charact', 'develop', 'ever', 'see'], ['certain', 'peopl', 'relat'], ['perfect', 'everi', 'aspect'], ['hype'], [], ['exist'], ['go', 'second', 'time', 'watch'], ['rather', 'superhero'], ['good', 'dark', 'suspen'], ['review'], ['real', 'life', 'joker'], ['final', 'real', 'movi'], ['good'], ['feed', 'cgi', 'fuel', 'faux', 'drama'], [], ['critic', 'useless'], [], ['would', 'call'], ['good', 'act', 'perform', 'have', 'ever', 'see'], ['forget', 'smile'], ['film'], ['extrem', 'overr'], [], [], ['great', 'know'], ['life'], ['extrem', 'overr'], ['requiem', 'broken', 'man'], ['rise'], ['brilliant', 'best', 'joker', 'have', 'ever', 'see'], [], ['old', 'peopl', 'get', 'stupid'], [], ['probabl', 'good', 'comic', 'book', 'perform', 'make', 'speechless', 'end', 'venic'], ['perform', 'have', 'come', 'know'], ['see', 'tonight', 'make', 'account', 'tell', 'everyon', 'good'], [], [], ['anyon', 'rate', 'cinema'], [], ['best', 'movi', 'ever'], [], ['a

In [0]:
import gensim.corpora as corpora
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)], [(10, 1), (11, 1), (12, 1)], [(13, 1), (14, 1), (15, 1)], [(2, 1)], [], [(16, 1)], [(1, 1), (17, 1), (18, 1), (19, 1)], [(20, 1), (21, 1)], [(7, 1), (22, 1), (23, 1)], [(24, 1)], [(25, 1), (26, 1), (27, 1)], [(27, 1), (28, 1), (29, 1)], [(7, 1)], [(30, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [], [(35, 1), (36, 1)], [], [(37, 1), (38, 1)], [(6, 1), (7, 1), (8, 1), (9, 1), (39, 1), (40, 1)], [(41, 1), (42, 1)], [(43, 1)], [(44, 1), (45, 1)], [], [], [(46, 1), (47, 1)], [(26, 1)], [(44, 1), (45, 1)], [(48, 1), (49, 1), (50, 1)], [(51, 1)], [(6, 1), (9, 1), (25, 1), (40, 1), (52, 1), (53, 1)], [], [(11, 1), (54, 1), (55, 1), (56, 1)], [], [(7, 1), (8, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1)], [(8, 1), (40, 1), (47, 1), (64, 1)], [(7, 1), (9, 1), (60, 1), (65, 1), (66, 1), (67, 1), (68, 1)], [], [], [(69, 1), (70, 1), (71, 1)], [], [(6, 1), (29, 1), (52, 1)], [], [(72, 1)], [(66, 

In [0]:
from pprint import pprint
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
pprint(lda_model.print_topics(num_words=4))
doc_lda = lda_model[corpus]

[(0, '0.112*"real" + 0.112*"perfect" + 0.112*"joke" + 0.059*"aspect"'),
 (1, '0.064*"good" + 0.064*"time" + 0.043*"make" + 0.043*"movi"'),
 (2, '0.097*"everyon" + 0.051*"feel" + 0.051*"broken" + 0.051*"wash"'),
 (3, '0.151*"good" + 0.062*"bad" + 0.062*"film" + 0.033*"best"'),
 (4, '0.080*"superhero" + 0.080*"rather" + 0.080*"rise" + 0.007*"good"'),
 (5, '0.089*"overr" + 0.089*"extrem" + 0.046*"know" + 0.046*"perform"'),
 (6, '0.032*"account" + 0.032*"tonight" + 0.032*"tell" + 0.032*"faux"'),
 (7, '0.093*"good" + 0.048*"ever" + 0.048*"peopl" + 0.048*"perform"'),
 (8, '0.089*"hype" + 0.061*"good" + 0.060*"believ" + 0.032*"film"'),
 (9, '0.085*"realli" + 0.085*"life" + 0.045*"go" + 0.045*"disappoint"')]


**1.Features used for Topic Modelling**                                         
1.Tokenizing the sentences into list of words and removing the stopwords, unnecesarry characters if any.                                                  
2.Gensim's Phrases Model can implement the bigrams,trigrams,quadgrams and more.The two important argument for the phrases are min_count and threshold.If the threshold is higher we get fewer phrases.                                       
3.The two main inputs of the LDA topic model are dictionary and the corpus.Gensim creates a unique id for each word in the document.The produced corpus shown above is a mapping of(word_id,word_frequency).                            
4.**LDA MODEL:**After creating the dictionary and corpus,we need to provide the num_topics as well.                                                              
I used chucksize and passes while building LDA model using gensims.Chunksize decides how many documents are processed at a time in the training algorithm and passes decides how often we can train the model on the entire corpus.                     
5.The above LDA model is build with the top 10 topics where each topic is the combination of keywords and each keyword contributes a certain weightage to the topic.                                                                           
lda_model.print_topics(num_words=4) is used to print the keywords of each topic and the importance(weightage) of each keyword with 4 topic terms for each topic.

**2.Top 10 Clusters for topic Modeling**                                        
The top 10 clusters obtained are:                                              
(0, '0.112*"real" + 0.112*"perfect" + 0.112*"joke" + 0.059*"aspect"'),      
 (1, '0.064*"good" + 0.064*"time" + 0.043*"make" + 0.043*"movi"'),      
 (2, '0.097*"everyon" + 0.051*"feel" + 0.051*"broken" + 0.051*"wash"'),     
 (3, '0.151*"good" + 0.062*"bad" + 0.062*"film" + 0.033*"best"'),    
 (4, '0.080*"superhero" + 0.080*"rather" + 0.080*"rise" + 0.007*"good"'),    
 (5, '0.089*"overr" + 0.089*"extrem" + 0.046*"know" + 0.046*"perform"'),   
 (6, '0.032*"account" + 0.032*"tonight" + 0.032*"tell" + 0.032*"faux"'),   
 (7, '0.093*"good" + 0.048*"ever" + 0.048*"peopl" + 0.048*"perform"'),     
 (8, '0.089*"hype" + 0.061*"good" + 0.060*"believ" + 0.032*"film"'),  
 (9, '0.085*"realli" + 0.085*"life" + 0.045*"go" + 0.045*"disappoint"')]

 **3.Summarize and describe the topics for each cluster**  
 0.Joker is a real hype with every perfect aspect.                             
 1.Spend good time and best movie making.                                        
 2.Everyone feel like broken and brain wash.                                     
 3.Bad film and insult to best comic book.                                       
 4.Psychological study rather than a superhero.                                     
 5.Over extreme performance.                                                     
 6.Saw tonight and made an account to tell everyone about faux drama.            
 7.One act performance of people I have ever seen.                               
 8.Believe the hype,Good film to watch.                                          
 9.Went blind dint enjoy that life of joker really disappointed.

 





# **Question 2: Sentiment Analysis**

(30 points). Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: **80% data for training and 20% data for testing**.  

(1) Features used for sentiment classification and explain why you select these features.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively. 

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9. 

In [0]:
# Write your code here
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv('Assignment5_File_5731.csv',encoding = "ISO-8859-1")
X = df['Clean_Title']
y = df['Sentiment']

In [33]:
#LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

vect = CountVectorizer(min_df=2, ngram_range=(1, 2))
X_train = vect.fit(X_train).transform(X_train) 
X_test = vect.transform(X_test)
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred,average = 'macro'))
print("Recall:",metrics.recall_score(y_test, y_pred,average = 'macro'))
print("f1 score:",metrics.f1_score(y_test, y_pred,average = 'macro'))

Accuracy: 0.7
Precision: 0.47222222222222215
Recall: 0.48888888888888893
f1 score: 0.47771836007130125


In [34]:
#support Vector Machine
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
vect = CountVectorizer(min_df=2, ngram_range=(1, 2))
X_train = vect.fit(X_train).transform(X_train) 
X_test = vect.transform(X_test)
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred,average = 'macro'))
print("Recall:",metrics.recall_score(y_test, y_pred,average = 'macro'))
print("f1 score:",metrics.f1_score(y_test, y_pred,average = 'macro'))

Accuracy: 0.5
Precision: 0.3790849673202614
Recall: 0.36296296296296293
f1 score: 0.30769230769230765


**1.Features used for Sentiment Classification**
A feature or aspect is an attribute of entity.Different features can generate different sentiment analysis.
The features used for sentiment classification are clean titles of the joker movie reviews and sentiment i.e. if the title is positive,negative or neutral.I chose these features to obtain the better accuarcy because the accuracy depends on the relatinship between the features.

For feature selection we need to divide the dataset into dependent and independent variables.Here dependent is the sentiment columns and independent is the clean title from our dataset.

Some of the techniques used for feature selection are Mutual Information,Chi-Square,Information gain and TF-idf to select geatures from high dimentionality of feature set.

**Classification of sentiment:**
![alt text](https://www.kdnuggets.com/images/sentiment-fig-2-532.jpg)


***1.LogisticRegression***
Logistic Regression is one of the most simple and commonly used Machine Learning algorithms for two-class classification.It describes and estimates the relationship between one dependent binary variable and independent variables.    
1.I have dividied the columns into two types variables dependent(y) and independent variable(X).
2.splitting the dataset into 80% training data and 20% test data using function train_test_split().In this function we need to pass three parameters dependent variable,independent variable and test_size and random_state can be used to select the records randomly.
3.LogisticRegression() function is used to create a Logistic Regression classifier object.
4.Fit() function is used to fit the model on the train set and predict() function is used to make predictions on the test set.
5.Accurcay,Precision,recall and f1 score are calculated to evaluate the metrics.                               
***2.Support Vector Machine***                                                   
SVM constructs a hyperplane in multidimensional space to separate different classes. SVM generates optimal hyperplane in an iterative manner, which is used to minimize an error. The core idea of SVM is to find a maximum marginal hyperplane(MMH) that best divides the dataset into classes.
1.I have dividied the columns into two types variables dependent(y) and independent variable(X).
2.splitting the dataset into 80% training data and 20% test data using function train_test_split().In this function we need to pass three parameters dependent variable,independent variable and test_size and random_state can be used to select the records randomly.
3.SVC() function is used to create a Support Vector classifier object by passing an argument kernel as the linear kernel.A linear Kernel is nothing but a dot product of any two given observations.
4.Fit() function is used to fit the model on the train set and predict() function is used to make predictions on the test set.
5.Accurcay,Precision,recall and f1 score are calculated to evaluate the metrics.

**3.Comparing the algorithms based on accuracy,precision,recall amd f1 score**

I have chosen the logistic regression and support vector machine algorithms because our dataset is very small and for accurate results over a small dataset we have few algorithms like logistic regression, SVM using linear, Naive Bayes, RandomForest.

From the baove results based on the accuracy,precision,recall and f1 score we can say that logistic regression classifier is best suited for this dataset as the accuracy is 70% which is good enough and the f1 score is close to 0.5 which means the algorithm is good for this dataset.


# **Question 3: House price prediction**

(40 points). You are required to build a **regression** model to predict the house price with 79 explanatory variables describing (almost) every aspect of residential homes. The purpose of this question is to practice regression analysis, an supervised learning model. The training data, testing data, and data description files can be download here: https://github.com/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/assignment4-question3-data.zip. Here is an axample for implementation: https://towardsdatascience.com/linear-regression-in-python-predict-the-bay-areas-home-price-5c91c8378878. 


In [36]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_X = train.drop('SalePrice', axis=1)
train_y = train.SalePrice
test_X = test
onehot_train_X = pd.get_dummies(train_X)
onehot_test_X = pd.get_dummies(test_X)
train_X, test_X = onehot_train_X.align(onehot_test_X, join='left', axis=1)
my_imputer = SimpleImputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)
reg = LinearRegression()
cv_scores = cross_val_score(reg, train_X, train_y, cv=5)
reg = LinearRegression()
reg.fit(train_X, train_y)
predictions = reg.predict(test_X)
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice':predictions})
my_submission.to_csv("Predicted_House_Price.csv")
my_submission

,Id,SalePrice
0,1461,112709.903813
1,1462,161010.363863
2,1463,186961.179807
3,1464,197300.132022
4,1465,205541.292943
...,...,...
1454,2915,86095.528874
1455,2916,81610.279372
1456,2917,181080.339264
1457,2918,117021.339522


In [37]:
#Performance evaluation for the baove model.
reg = LinearRegression()
reg.fit(train_X,train_y)
X_train , X_test, Y_train, Y_test = train_test_split(
    train_X,train_y,
        test_size=0.20,
        random_state=42)
predictions = reg.predict(X_test)
print("Mean Absolute Error", metrics.mean_absolute_error(Y_test, predictions))
print("Mean Square Error",metrics.mean_squared_error(Y_test, predictions))
print("R2 Score",metrics.r2_score(Y_test, predictions))

Mean Absolute Error 14426.740961107349
Mean Square Error 522409274.01144296
R2 Score 0.931892169915545
